In [1]:
# %load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os
import sys
import time
import pdb
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import networkx as nx

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.model_selection import train_test_split

# Clustering
from sklearn.datasets import make_biclusters, make_checkerboard, make_classification
from sklearn.datasets import samples_generator as sg
from sklearn.cluster.bicluster import SpectralCoclustering, SpectralBiclustering
from sklearn.metrics import consensus_score

# Corr
from scipy.stats import spearmanr, pearsonr
from numpy import corrcoef

# Utils
file_path = os.getcwd()  # os.path.dirname(os.path.relpath(__file__))
utils_path = os.path.abspath(os.path.join(file_path, '..', '..', 'utils_py'))
sys.path.append(utils_path)
import utils_all as utils

# Feature importance
pfi_path = os.path.abspath(os.path.join(file_path, '..', 'pfi'))
sys.path.append(pfi_path)
import pfi
import pfi_utils

import warnings
warnings.filterwarnings('ignore')

SEED=0

Using TensorFlow backend.


In [4]:
APP = 'tc'

In [6]:
DATAPATH_TR = os.path.join(file_path, 'data', f'{APP}_data_train_raw')
DATAPATH_VL = os.path.join(file_path, 'data', f'{APP}_data_val_raw')
YENC_PATH = os.path.join(file_path, 'data', f'{APP}_y_enc')

In [7]:
y_enc = pd.read_csv(YENC_PATH, sep='\t')
data_train = pd.read_csv(DATAPATH_TR, sep='\t')
data_val = pd.read_csv(DATAPATH_VL, sep='\t')
print(f'\ndata_train.shape {data_train.shape}')
print(f'data_val.shape   {data_val.shape}')


data_train.shape (4080, 942)
data_val.shape   (1020, 942)


In [9]:
xtr = data_train.iloc[:, 1:].copy()
ytr = data_train.iloc[:, 0].copy()
xvl = data_val.iloc[:, 1:].copy()
yvl = data_val.iloc[:, 0].copy()
features = xtr.columns

In [18]:
tmp = pd.concat([yvl, xvl], axis=1)
tmp = tmp.sort_values(by='y', axis=0).reset_index(drop=True)
ydata = tmp.iloc[:,0].copy()
xdata = tmp.iloc[:,1:].copy()

In [19]:
xdata

,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,ACAT2,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
0,-0.118141,-1.306469,0.488940,0.226565,0.283409,-0.935897,-0.635514,-0.559602,0.282940,-0.229867,...,0.374783,1.796675,-0.138344,0.557359,-0.760372,0.193864,-0.579828,0.969928,-0.770174,-0.789771
1,-0.306094,-0.187702,0.056925,0.641482,-0.089469,-1.739043,-1.096865,-2.277652,-0.669335,-0.470464,...,-1.858819,-1.703501,-1.585435,-2.204703,-1.981818,0.183216,-1.347292,-1.074255,0.148996,-0.421560
2,1.174033,-0.043702,0.164928,0.892946,0.856043,1.341515,-0.242009,0.519504,2.294621,1.463565,...,-0.854953,1.220515,0.076764,0.515719,0.994579,0.460053,0.134092,1.132533,-0.303055,1.435507
3,0.304752,0.288605,1.253966,0.025393,0.536433,1.385523,-0.296286,-0.729987,1.044760,0.464163,...,1.353552,1.436575,0.428759,0.071569,-0.704214,-0.892187,-0.615524,0.203359,-0.061961,0.699084
4,-1.175375,0.321835,0.515941,-0.804441,-1.421177,0.010274,-0.947604,-1.141751,-0.002742,-1.294045,...,0.249300,-0.075847,-0.744558,-1.968748,-0.704214,-0.082973,-0.383500,-0.145081,-0.483876,-0.773762
5,-0.411817,-0.021548,-0.924109,-0.276365,0.256775,0.395344,-1.137572,-0.914571,0.294844,-0.109568,...,-1.557659,-0.219887,-0.275231,0.890471,-0.479580,-0.114915,0.169788,-0.516751,-0.649628,0.811148
6,-0.247359,-0.763701,-0.042079,0.440310,-0.129420,0.340334,-0.119887,-0.318223,-0.252715,-1.506881,...,0.675943,0.586738,0.819865,0.432441,-0.254946,0.012855,0.687380,0.644717,0.359953,0.971240
7,0.950839,-0.608624,-0.402091,0.616335,0.536433,0.428350,-0.934035,-0.176235,0.009161,0.371626,...,-0.854953,-0.608796,-0.333897,-0.497499,-0.928848,0.161921,0.062700,-0.679356,0.465432,-0.021330
8,-0.294347,-0.187702,0.695947,0.603762,0.856043,-0.187762,0.124357,0.150336,1.568511,0.223566,...,0.776329,1.955120,1.250081,0.196486,0.489153,0.449405,1.133580,1.388056,-0.001688,-0.533624
9,-0.775976,-0.553240,0.155928,0.163699,-0.515616,0.087288,-0.974743,-0.261428,0.604333,-1.053448,...,-1.143565,-0.421543,-0.412118,-0.580777,-0.605937,-0.594056,-0.883244,-0.028934,-0.393465,-0.501606


In [2]:
df_rna, meta = utils.load_lincs1000(dataset='raw', sources=['gdc'])
df_rna[:3]


Dataset: combined_rnaseq_data_lincs1000
df_rna (11081, 943)
meta   (11081, 20)


,Sample,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
0,GDC.ADR_ACC.TCGA_OR_A5J1_08081.p,3.76,3.11,1.37,2.94,7.21,3.58,0.61,1.38,2.77,...,0.45,1.53,0.48,1.24,0.77,2.09,1.32,0.23,0.65,0.84
1,GDC.ADR_ACC.TCGA_OR_A5J2_10193.p,5.02,1.65,2.09,3.99,6.57,3.75,2.99,1.84,3.38,...,2.08,2.32,0.84,2.37,1.94,1.77,1.25,0.95,1.20,1.28
2,GDC.ADR_ACC.TCGA_OR_A5J3_09853.p,4.33,2.47,1.77,3.29,5.85,5.42,1.49,2.05,2.98,...,1.46,1.72,0.94,1.61,1.38,2.53,1.16,0.18,0.73,1.20


In [3]:
dd = pd.read_csv('/vol/ml/apartin/projects/feature-importance/apps/data/P1B1.dev.train.lincs', sep='\t')
dd[:3]

,y,AARS,ABCB6,ABCC5,ABCF1,ABCF3,ABHD4,ABHD6,ABL1,ACAA1,...,ZMIZ1,ZMYM2,ZNF131,ZNF274,ZNF318,ZNF395,ZNF451,ZNF586,ZNF589,ZW10
0,3,19.3,15.6,16.3,18.9,17.8,16.9,15.2,18.0,17.3,...,17.1,17.3,16.1,17.0,15.9,16.2,16.2,16.0,16.5,17.2
1,0,20.2,15.6,16.3,18.8,18.1,18.5,18.3,18.9,18.2,...,19.0,16.7,16.2,16.5,16.2,17.8,15.8,14.9,16.7,16.5
2,9,19.6,14.1,16.3,18.8,17.7,18.7,15.5,18.5,17.0,...,17.6,17.8,16.8,15.9,16.0,16.7,15.7,16.1,16.5,17.5
